# Overview

The Semantic Router algorithm converts all the training texts to embeddings using any off-the-shelf embedding models like Sentence Transformer. During inference, it converts the input texts to embeddings and finds the K Nearest Neighbots from the training data embeddings to classify the input text.

In [8]:
!pip install -q -U transformers==4.39.3
!pip install -q -U datasets==2.18.0
!pip install -q -U semantic-router==0.0.44

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.5.3 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.39.3 requires tokenizers<0.19,>=0.14, but you have tokenizers 0.19.1 which is incompatible.


In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["MODEL"]="sentence-transformers/paraphrase-mpnet-base-v2"
os.environ["DATASET"]="SetFit/amazon_massive_intent_en-US"
os.environ["FITMODEL"]="semantic-router-mpnet-v2-amazon-mi"

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading dataset

In [3]:
from datasets import Dataset, load_dataset

ds=load_dataset(os.getenv("DATASET"))

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Pre-processing dataset

In [4]:
import pandas as pd

df=pd.DataFrame(ds["test"])

# Helper function to select random rows
def select_random_rows(group):
    return group.sample(n=10, random_state=42)


# find top classes with minimum 30 rows
label_counts=df["label_text"].value_counts()
label_counts=label_counts[label_counts>30]

# restruct df to top classes only
df=df[df["label_text"].isin(label_counts.index)].reset_index(drop=True)

assert set(df["label_text"].value_counts().index.to_list())==set(label_counts.index.to_list()), "Some labels were lost"

# select random row per unique value in label_text column
train_df=df.groupby("label_text", group_keys=False).apply(select_random_rows)

# create eval dataframe by dropping the train data and selecting random rows
eval_df=(df.drop(train_df.index).groupby("label_text",group_keys=False).apply(select_random_rows))

# create test dataframe by dropping both train and eval data
test_df=df.drop(train_df.index.to_list()+eval_df.index.to_list())

# reset the index
cols_to_keep=["text", "label_text"]
train_df=train_df[cols_to_keep].reset_index(drop=True)
eval_df=eval_df[cols_to_keep].reset_index(drop=True)
test_df=test_df[cols_to_keep].reset_index(drop=True)

# save the file
test_df.to_pickle("test_df.pkl")
train_df.to_pickle("train_df.pkl")
eval_df.to_pickle("eval_df.pkl")

train_ds=Dataset.from_pandas(train_df)
eval_ds=Dataset.from_pandas(eval_df)
test_ds=Dataset.from_pandas(test_df)

print(train_df.shape, eval_df.shape, test_df.shape)
train_df.head()

(350, 2) (350, 2) (1879, 2)


/tmp/ipykernel_34/2096072177.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df=df.groupby("label_text", group_keys=False).apply(select_random_rows)
/tmp/ipykernel_34/2096072177.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eval_df=(df.drop(train_df.index).groupby("label_text",group_keys=False).apply(select_random_rows))


,text,label_text
0,do i have any alarms set for six am tomorrow,alarm_query
1,what is the wake up time for my alarm i have s...,alarm_query
2,please tell me what alarms are on,alarm_query
3,please list all my alarms,alarm_query
4,what times do my alarms go off,alarm_query


# Training

In [9]:
from semantic_router import Route
from semantic_router.encoders import HuggingFaceEncoder
from semantic_router.layer import RouteLayer

routes=[]

for topic in train_df["label_text"].unique():
    name=topic
    utterances = train_df[train_df["label_text"] == topic]["text"].values.tolist()
    route = Route(name=name, utterances=utterances)
    routes.append(route)

# Step 2: Setup the Embedding model
encoder = HuggingFaceEncoder(name="sentence-transformers/all-MiniLM-L6-v2")

# Step 3: Create the route layer to embed all utterences and assign their class
rl = RouteLayer(encoder=encoder, routes=routes)   

ImportError: Please install transformers to use HuggingFaceEncoder. You can install it with: `pip install semantic-router[local]`

# Inference

In [ ]:
from sklearn.metrics import classification_report
from tdqm import tdqm

# Step 1: Run predictions to calculate class level metrics
predictions = []
for text in tqdm(test_df['text'].values):
    pred = rl(text).name

    if pred:
        predictions.append(pred)
    else:
        predictions.append("")

test_df["semantic_router_predictions"] = predictions
print("\nSemantic Router Class Level Metrics:")
print(
    classification_report(test_df["label_text"], test_df["semantic_router_predictions"])
)

# Acknowledge

* https://medium.com/towards-artificial-intelligence/few-shot-nlp-intent-classification-d29bf85548aa
* https://github.com/aurelio-labs/semantic-router/tree/main